In [20]:
import pandas as pd
import numpy as np


## Reading relevant data
match_data = pd.read_csv('./match_data.csv')
batsman_lvl_data = pd.read_csv('./batsman_data.csv')
bowler_lvl_data = pd.read_csv('./bowler_data.csv')
train_data = pd.read_csv('./df_train.csv')
test_data = pd.read_csv('./df_test.csv')

In [21]:
# Function to get player IDs
def get_player_ids(roster_id_str):
    players_id = [int(float(x)) for x in roster_id_str.split(":")]
    return players_id

In [22]:
pd.set_option('display.max_columns',0)
pd.set_option('display.max_colwidth',0)

# Batsmen feature generation

In [23]:
# Function to calculate team batsmen average strike rate of last 5 matches
def team_bats_avg_strike_last5(team_roster_ids, match_dates, batsman_lvl_data):
    team_bats_avg_strike = []

    for tm_roster_id, match_date in zip(team_roster_ids, match_dates):
        players = get_player_ids(tm_roster_id)
        bats_indices = [i for i, player in enumerate(players) if player in batsman_lvl_data['batsman_id'].values]
        bats_strike = []

        for i in bats_indices:
            my_batsid = players[i]
            rdat = batsman_lvl_data[(batsman_lvl_data['batsman_id'] == my_batsid) & (batsman_lvl_data['match_dt'] < match_date)]
            rdat = rdat.sort_values('match_dt')
            rdat_len = len(rdat)

            if rdat_len >= 5:
                rdat_last5 = rdat.tail(5)
            elif rdat_len == 0:
                bats_strike.append(0)
                continue
            else:
                rdat_last5 = rdat.tail(rdat_len)

            bats_strike.append(rdat_last5['strike_rate'].mean())

        team_bats_avg_strike.append(np.mean(bats_strike))

    return team_bats_avg_strike


In [24]:
# Train data feature
train_data['team1_bats_avg_strike'] = team_bats_avg_strike_last5(train_data['team1_roster_ids'], train_data['match_dt'], batsman_lvl_data)
train_data['team2_bats_avg_strike'] = team_bats_avg_strike_last5(train_data['team2_roster_ids'], train_data['match_dt'], batsman_lvl_data)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [25]:
train_data.head()

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,team1_bats_avg_strike,team2_bats_avg_strike
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:3299407.0:232000.0:7883504.0:8725723.0:7878989.0:2526390.0:8725730.0:4017523.0,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:3698337.0:5058192.0:6844718.0:4566526.0:6596582.0:4161975.0:7952804.0:8465057.0,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.0,157.178571,53.048286,71.956000
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409.0:172199.0:2022957.0:1482998.0:1711492.0:4507208.0:3231437.0,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:1613926.0:2740408.0:4739552.0:5744850.0:4690104.0:3715697.0:3468870.0,Ed,20,Wt Is,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.0,103.500000,106.552955,114.335591
2,9433269,We,10576,3298427.0:2288789.0:7773338.0:3519011.0:3681957.0:7671313.0:5320118.0:2922611.0:7184232.0:7661996.0:6240632.0,Ne,8987,4003390.0:1749075.0:1626526.0:4172447.0:5516720.0:7328110.0:125915.0:130437.0:7507303.0:4420828.0:5652828.0,We,10576,Ne,field,Tt Be Nm,Nottingham,2023-06-02,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.0,154.333333,100.919182,95.301303
3,9587073,Ga An Ws,36084,8127230.0:4690328.0:4069666.0:7960847.0:4690188.0:4739580.0:4739552.0:5744850.0:2294823.0:9266102.0:284339.0,Bs Rs,36070,3462080.0:2436405.0:1798705.0:7550857.0:5742470.0:2740408.0:3575774.0:3200756.0:334963.0:6399210.0:6347494.0,Ga An Ws,36084,Ga An Ws,bat,Bn La Sm Ta Td,Tarouba,2023-09-10,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.0,144.250000,101.380364,88.362727
4,9516457,Pb Ks,30407,8127181.0:197658.0:4239038.0:2398346.0:5053082.0:4640824.0:5038046.0:8180500.0:2035102.0:3851518.0:7881845.0:7453697.0,Gt Ts,48341,1958683.0:7491224.0:8059029.0:4377610.0:2252452.0:2966879.0:5554254.0:4690230.0:3373285.0:3759846.0:6496482.0,Gt Ts,48341,Gt Ts,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,2023-04-13,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.0,189.000000,119.846182,101.171400


In [26]:
# Test data feature
test_data['team1_bats_avg_strike'] = team_bats_avg_strike_last5(test_data['team1_roster_ids'], test_data['match_dt'], batsman_lvl_data)
test_data['team2_bats_avg_strike'] = team_bats_avg_strike_last5(test_data['team2_roster_ids'], test_data['match_dt'], batsman_lvl_data)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [27]:
test_data.head()

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,team1_bats_avg_strike,team2_bats_avg_strike
0,9250275,Jy,28594,7438955.0:8271969.0:8369661.0:3685247.0:2590251.0:7785497.0:7785553.0:3296264.0:4613622.0:2904796.0:3685289.0,Ud Ss of Aa,90,2436944.0:8117500.0:6703528.0:3010748.0:1613870.0:8934764.0:2721480.0:1632819.0:2789604.0:6489846.0:1565507.0,Ud Ss of Aa,field,Bo Ac Cb,Bulawayo,2022-07-11,day match,Ud Ss of Aa tr of Ze,2022,3226,0.000000,0.019608,NaN,0.00,NaN,24.811250,82.637273
1,9262189,Ga An Ws,36084,3715690.0:6818622.0:4069666.0:393014.0:4690188.0:4739552.0:4690328.0:5744850.0:4739580.0:9266102.0:284339.0,Ja Ts,36098,4690258.0:3761246.0:8464385.0:5742470.0:3150720.0:4690104.0:181404.0:1594319.0:5509524.0:4239794.0:2036649.0,Ja Ts,field,Pe Sm Ga,Providence,2022-09-21,night match,Cn Pr Le,2022,13915,0.615385,0.344262,151.285714,66.67,153.500,104.434182,106.737455
2,9128776,Rn Rs,30428,8058959.0:2162782.0:2981614.0:7833195.0:7556051.0:2486896.0:184960.0:1945397.0:6420126.0:3011735.0:6347494.0,Lw Sr Gs,48334,2654014.0:2954769.0:8058903.0:3479860.0:3299407.0:2275097.0:2740408.0:3865077.0:7924048.0:4859490.0:8228100.0,Rn Rs,bat,Be Sm Mi,Mumbai,2022-05-15,night match,In Pr Le,2022,2764,0.842105,0.753086,171.066667,100.00,179.625,95.311030,101.803091
3,9586919,Ja Ts,36098,4690258.0:8464385.0:1613898.0:5744780.0:3150720.0:4690104.0:1594319.0:5509524.0:2036649.0:3023383.0:5471948.0,St La Ks,36112,2331475.0:313809.0:391103.0:2742837.0:2097017.0:7921535.0:7459647.0:8464343.0:4690230.0:8339701.0:4702774.0,St La Ks,field,Dn Sy Nl Ct Sm Gs It St La,Gros Islet,2023-08-16,night match,Cn Pr Le,2023,11892,0.285714,1.487805,166.000000,75.00,155.500,95.643778,101.966667
4,9128538,Ci Sr Ks,30414,7422673.0:249087.0:62432.0:232000.0:5001170.0:1642738.0:196580.0:360086.0:2022957.0:7968225.0:7879829.0,Gt Ts,48341,1958683.0:7491224.0:3339160.0:5452754.0:2252452.0:2966879.0:5554254.0:4690230.0:3456424.0:3373285.0:8118053.0,Gt Ts,field,Ma Ct An Sm Pe,Pune,2022-04-17,night match,In Pr Le,2022,18752,2.375000,0.310330,169.933333,0.00,164.125,113.618455,106.104700


# Bowler feature generation

In [28]:
# Function to calculate performance score
def calculate_performance_score(runs, wicket_count, balls_bowled, economy, maiden, dots, Fours, Sixes, wides, noballs):
    weight_wickets = 25
    weight_maidens = 5
    weight_dots = 2
    weight_runs = -1
    weight_fours = -0.5
    weight_sixes = -0.8
    weight_wides = -0.2
    weight_noballs = -0.4

    performance_score = (wicket_count * weight_wickets) + (maiden * weight_maidens) + (dots * weight_dots) + (runs * weight_runs) + (Fours * weight_fours) + (Sixes * weight_sixes) + (wides * weight_wides) + (noballs * weight_noballs)

    return performance_score


In [29]:
def team_bowler_feat_last5(team_roster_ids, match_dates, bowler_lvl_data):
    team_bowlers_avg_perfor = []
    team_bowlers_avg_eco = []

    for tm_roster_id, match_date in zip(team_roster_ids, match_dates):
        players = get_player_ids(tm_roster_id)
        bowler_indices = [i for i, player in enumerate(players) if player in bowler_lvl_data['bowler_id'].values]
        bowler_summary = []
        bowler_economy = []

        for i in bowler_indices:
            my_bowlerid = players[i]
            rdat = bowler_lvl_data[(bowler_lvl_data['bowler_id'] == my_bowlerid) & (bowler_lvl_data['match_dt'] < match_date)]
            rdat = rdat.sort_values('match_dt')
            rdat_len = len(rdat)

            if rdat_len >= 5:
                rdat_last5 = rdat.tail(5)
            elif rdat_len == 0:
                bowler_summary.append(0)
                bowler_economy.append(0)
                continue
            else:
                rdat_last5 = rdat.tail(rdat_len)

            rdat_last5 = rdat_last5.copy()  # Create a copy to avoid the SettingWithCopyWarning
            rdat_last5['performance_score'] = rdat_last5.apply(lambda row: calculate_performance_score(row['runs'], row['wicket_count'], row['balls_bowled'], row['economy'], row['maiden'], row['dots'], row['Fours'], row['Sixes'], row['wides'], row['noballs']), axis=1)

            bowler_summary.append(rdat_last5['performance_score'].mean())
            bowler_economy.append(rdat_last5['economy'].mean())

        team_bowlers_avg_perfor.append(np.mean(bowler_summary) if bowler_summary else 0)
        team_bowlers_avg_eco.append(np.mean(bowler_economy) if bowler_economy else 0)

    overall_avg_perfor = np.mean([x for x in team_bowlers_avg_perfor if x != 0])
    overall_avg_eco = np.mean([x for x in team_bowlers_avg_eco if x != 0])

    team_bowlers_avg_perfor = [x if x != 0 else overall_avg_perfor for x in team_bowlers_avg_perfor]
    team_bowlers_avg_eco = [x if x != 0 else overall_avg_eco for x in team_bowlers_avg_eco]

    return team_bowlers_avg_perfor, team_bowlers_avg_eco


In [30]:
# Train data feature
team1_bowlers_features = team_bowler_feat_last5(train_data['team1_roster_ids'], train_data['match_dt'], bowler_lvl_data)
team2_bowlers_features = team_bowler_feat_last5(train_data['team2_roster_ids'], train_data['match_dt'], bowler_lvl_data)

train_data["tm1_eco_p_tm2_perfor"] = [i/j for i, j in zip(team1_bowlers_features[0], team2_bowlers_features[0])]
train_data["tm1_eco_p_tm2_eco"] = [i/j for i, j in zip(team1_bowlers_features[1], team2_bowlers_features[1])]


In [31]:
train_data.head()

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,winner,winner_id,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,team1_bats_avg_strike,team2_bats_avg_strike,tm1_eco_p_tm2_perfor,tm1_eco_p_tm2_eco
0,9331181,Ba,11283,9373356.0:7857520.0:4232164.0:4566540.0:3299407.0:232000.0:7883504.0:8725723.0:7878989.0:2526390.0:8725730.0:4017523.0,Hl Ph,12634,3500958.0:4231751.0:2735081.0:2035102.0:3698337.0:5058192.0:6844718.0:4566526.0:6596582.0:4161975.0:7952804.0:8465057.0,Hl Ph,12634,Hl Ph,field,Hr Ct Sm Ie,Indore,2022-10-20,day/night match,Sd Mq Ai Ty,2022/23,7398,1.666667,0.672131,139.000000,100.0,157.178571,53.048286,71.956000,0.792797,0.866174
1,8797060,Ed,20,2089079.0:6139370.0:2076192.0:62432.0:2083409.0:172199.0:2022957.0:1482998.0:1711492.0:4507208.0:3231437.0,Wt Is,41,4690258.0:4069666.0:4230127.0:1942317.0:1613926.0:2740408.0:4739552.0:5744850.0:4690104.0:3715697.0:3468870.0,Ed,20,Wt Is,field,Kn Ol Bn Bs,Bridgetown,2022-01-23,day/night match,Ed tr of Wt Is,2021/22,1406,1.285714,1.952381,156.000000,50.0,103.500000,106.552955,114.335591,2.808974,0.807064
2,9433269,We,10576,3298427.0:2288789.0:7773338.0:3519011.0:3681957.0:7671313.0:5320118.0:2922611.0:7184232.0:7661996.0:6240632.0,Ne,8987,4003390.0:1749075.0:1626526.0:4172447.0:5516720.0:7328110.0:125915.0:130437.0:7507303.0:4420828.0:5652828.0,We,10576,Ne,field,Tt Be Nm,Nottingham,2023-06-02,day/night match,Vy Bt,2023,251,0.857143,0.672131,173.266667,0.0,154.333333,100.919182,95.301303,0.802434,1.069791
3,9587073,Ga An Ws,36084,8127230.0:4690328.0:4069666.0:7960847.0:4690188.0:4739580.0:4739552.0:5744850.0:2294823.0:9266102.0:284339.0,Bs Rs,36070,3462080.0:2436405.0:1798705.0:7550857.0:5742470.0:2740408.0:3575774.0:3200756.0:334963.0:6399210.0:6347494.0,Ga An Ws,36084,Ga An Ws,bat,Bn La Sm Ta Td,Tarouba,2023-09-10,day match,Cn Pr Le,2023,14300,2.166667,1.975610,164.266667,50.0,144.250000,101.380364,88.362727,2.466747,0.840831
4,9516457,Pb Ks,30407,8127181.0:197658.0:4239038.0:2398346.0:5053082.0:4640824.0:5038046.0:8180500.0:2035102.0:3851518.0:7881845.0:7453697.0,Gt Ts,48341,1958683.0:7491224.0:8059029.0:4377610.0:2252452.0:2966879.0:5554254.0:4690230.0:3373285.0:3759846.0:6496482.0,Gt Ts,48341,Gt Ts,field,Pb Ct An IS Ba Sm Mi Ch,Chandigarh,2023-04-13,night match,In Pr Le,2023,7118,0.818182,1.327869,164.666667,0.0,189.000000,119.846182,101.171400,0.646214,0.987516


In [37]:
# Define your team_bowler_feat_last5 function as previously corrected

# Assuming test_data is a DataFrame and calculate_performance_score and get_player_ids functions are defined

team1_bowlers_features = team_bowler_feat_last5(test_data['team1_roster_ids'], test_data['match_dt'], bowler_lvl_data)
team2_bowlers_features = team_bowler_feat_last5(test_data['team2_roster_ids'], test_data['match_dt'], bowler_lvl_data)

test_data["tm1_eco_p_tm2_perfor"] = [i / j if j != 0 else np.nan for i, j in zip(team1_bowlers_features[0], team2_bowlers_features[0])]
test_data["tm1_eco_p_tm2_eco"] = [i / j if j != 0 else np.nan for i, j in zip(team1_bowlers_features[1], team2_bowlers_features[1])]

# Optional: Handle NaN values if division by zero occurred
test_data["tm1_eco_p_tm2_perfor"].fillna(0, inplace=True)
test_data["tm1_eco_p_tm2_eco"].fillna(0, inplace=True)



In [38]:
test_data.head()


,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,team1_bats_avg_strike,team2_bats_avg_strike,tm1_eco_p_tm2_perfor,tm1_eco_p_tm2_eco
0,9250275,Jy,28594,7438955.0:8271969.0:8369661.0:3685247.0:2590251.0:7785497.0:7785553.0:3296264.0:4613622.0:2904796.0:3685289.0,Ud Ss of Aa,90,2436944.0:8117500.0:6703528.0:3010748.0:1613870.0:8934764.0:2721480.0:1632819.0:2789604.0:6489846.0:1565507.0,Ud Ss of Aa,field,Bo Ac Cb,Bulawayo,2022-07-11,day match,Ud Ss of Aa tr of Ze,2022,3226,0.000000,0.019608,NaN,0.00,NaN,24.811250,82.637273,1.167018,1.852409
1,9262189,Ga An Ws,36084,3715690.0:6818622.0:4069666.0:393014.0:4690188.0:4739552.0:4690328.0:5744850.0:4739580.0:9266102.0:284339.0,Ja Ts,36098,4690258.0:3761246.0:8464385.0:5742470.0:3150720.0:4690104.0:181404.0:1594319.0:5509524.0:4239794.0:2036649.0,Ja Ts,field,Pe Sm Ga,Providence,2022-09-21,night match,Cn Pr Le,2022,13915,0.615385,0.344262,151.285714,66.67,153.500,104.434182,106.737455,1.133359,0.928929
2,9128776,Rn Rs,30428,8058959.0:2162782.0:2981614.0:7833195.0:7556051.0:2486896.0:184960.0:1945397.0:6420126.0:3011735.0:6347494.0,Lw Sr Gs,48334,2654014.0:2954769.0:8058903.0:3479860.0:3299407.0:2275097.0:2740408.0:3865077.0:7924048.0:4859490.0:8228100.0,Rn Rs,bat,Be Sm Mi,Mumbai,2022-05-15,night match,In Pr Le,2022,2764,0.842105,0.753086,171.066667,100.00,179.625,95.311030,101.803091,0.837286,1.123313
3,9586919,Ja Ts,36098,4690258.0:8464385.0:1613898.0:5744780.0:3150720.0:4690104.0:1594319.0:5509524.0:2036649.0:3023383.0:5471948.0,St La Ks,36112,2331475.0:313809.0:391103.0:2742837.0:2097017.0:7921535.0:7459647.0:8464343.0:4690230.0:8339701.0:4702774.0,St La Ks,field,Dn Sy Nl Ct Sm Gs It St La,Gros Islet,2023-08-16,night match,Cn Pr Le,2023,11892,0.285714,1.487805,166.000000,75.00,155.500,95.643778,101.966667,2.125587,0.960721
4,9128538,Ci Sr Ks,30414,7422673.0:249087.0:62432.0:232000.0:5001170.0:1642738.0:196580.0:360086.0:2022957.0:7968225.0:7879829.0,Gt Ts,48341,1958683.0:7491224.0:3339160.0:5452754.0:2252452.0:2966879.0:5554254.0:4690230.0:3456424.0:3373285.0:8118053.0,Gt Ts,field,Ma Ct An Sm Pe,Pune,2022-04-17,night match,In Pr Le,2022,18752,2.375000,0.310330,169.933333,0.00,164.125,113.618455,106.104700,0.393975,1.171551


In [39]:
train_data['winner_01'] = train_data.apply(lambda x: 0 if (x['team1']==x['winner']) else 1, axis=1)

In [41]:
from sklearn.model_selection import train_test_split

In [44]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [43]:
pip install lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 39.4 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lightgbm: filename=lightgbm-4.3.0-py3-none-linux_x86_64.whl size=2461330 sha256=d9e19678be330f0c49700d953dbae0112a569c254e15378ef541eed1ef4f3191
  Stored in directory: /home/ec2-user/.cache/pip/wheels/6b/92/ab/b7b5df76502b64443c1a830e5f7ec3cb66741313ddebb682aa
Successfully built lightgbm
Note: you may need to restart the kernel to use updated packages.


In [45]:
target = 'winner_01'  # Update with your actual target column name
features = ['team1_id', 'team2_id', 'tm1_eco_p_tm2_perfor', 'tm1_eco_p_tm2_eco',  
            'team_count_50runs_last15', 'team_winp_last5', 'team1only_avg_runs_last15', 
            'team1_winp_team2_last15', 'ground_avg_runs_last15', 'team1_bats_avg_strike', 
            'team2_bats_avg_strike']

In [48]:
X_train = train_data[features]
y_train = train_data[target]


In [49]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [50]:
train_data_lgb = lgb.Dataset(X_train, label=y_train)
val_data_lgb = lgb.Dataset(X_val, label=y_val, reference=train_data_lgb)

In [68]:
params = {
    'objective': 'binary',  # For binary classification
    'boosting_type': 'gbdt',
    'metric': 'binary_logloss',
    # 'num_leaves': 31,
    # 'learning_rate': 0.05,
    # 'feature_fraction': 0.9
}


In [69]:
gbm = lgb.train(params,
                train_data_lgb,
                num_boost_round=1000,
                # valid_sets=[train_data_lgb, val_data_lgb],
               )

[LightGBM] [Info] Number of positive: 386, number of negative: 372
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000335 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1889
[LightGBM] [Info] Number of data points in the train set: 758, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.509235 -> initscore=0.036944
[LightGBM] [Info] Start training from score 0.036944
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [70]:
y_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

In [71]:
accuracy = accuracy_score(y_val, y_pred_binary)
print(f'Accuracy: {accuracy}')
print(classification_report(y_val, y_pred_binary))

Accuracy: 0.4789473684210526
              precision    recall  f1-score   support

           0       0.50      0.45      0.48        99
           1       0.46      0.51      0.48        91

    accuracy                           0.48       190
   macro avg       0.48      0.48      0.48       190
weighted avg       0.48      0.48      0.48       190

